In [ ]:
import pickle

In [5]:
import pandas as pd
df = pd.read_csv('./sample_250_rec.csv')
df.head()

,storenbr,rxnbr,rfnbr,dispenseddrugstoreid,dispenseddrugkey,orderdate,filldate,dispensedqty1,rxqty,daysupp,...,SSRXNBR,Rx_key,FirstFillDate,LastFillDate,DaysWithNoMedicine,totalSupply,totalRefills,MPR1,abandonDays,MPR2
0,103237,446697,2,NaN,19279,17-04-2017,17-04-2017,120,30,40,...,145292,100285776,17-04-2017,08-06-2017,32,40,3,0.555556,1,0.547945
1,103237,446698,0,NaN,22771,17-04-2017,17-04-2017,300,60,150,...,145294,100285779,17-04-2017,08-06-2017,0,150,3,1.000000,0,1.000000
2,101867,1083372,3,NaN,24135,01-03-2019,04-03-2019,4,1,4,...,464187,100855210,04-03-2019,13-05-2019,87,4,4,0.043956,0,0.043956
3,103237,446679,2,NaN,25329,17-04-2017,17-04-2017,240,60,60,...,145289,100285771,17-04-2017,08-06-2017,22,60,3,0.731707,1,0.722892
4,103237,460259,2,NaN,7328,07-08-2017,01-09-2017,90,30,90,...,201637,100387953,01-09-2017,06-11-2017,6,90,3,0.937500,0,0.937500


In [6]:
with open('cat_model.pkl', 'rb') as f:
    pred_model = pickle.load(f)

In [7]:
test_data = df[['orderdate','filldate','miss_flag', 'dispensedqty1', 'rxqty', 'daysupp', 'refills', 'rxstatus', 'deliverystatus', 'drug_form', 'routeofadmin', 'sex', 'age', 'drug_name', 'tier', 'Disease', 'THERAPYCODE', 'HouseholdIncome', 'PLANID', 'PLANNAME', 'PAYORID', 'PROVIDERTYPE', 'Cashpaid', 'monthlyCoPay', 'UnitsPerMOnth', 'AffordabilityIndex', 'OrderMonth', 'orderweek','DaysWithNoMedicine', 'totalSupply', 'totalRefills', 'abandonDays']]

In [8]:
test_data.head()

,orderdate,filldate,miss_flag,dispensedqty1,rxqty,daysupp,refills,rxstatus,deliverystatus,drug_form,...,Cashpaid,monthlyCoPay,UnitsPerMOnth,AffordabilityIndex,OrderMonth,orderweek,DaysWithNoMedicine,totalSupply,totalRefills,abandonDays
0,17-04-2017,17-04-2017,1,120,30,40,5,0,0,TAB,...,83.52,62.64,90.0,0.0,4,16,32,40,3,1
1,17-04-2017,17-04-2017,1,300,60,150,5,0,0,CAP,...,1125.00,225.00,60.0,0.0,4,16,0,150,3,0
2,01-03-2019,04-03-2019,1,4,1,4,3,0,0,SOSY,...,26405.00,198037.50,30.0,0.0,3,9,87,4,4,0
3,17-04-2017,17-04-2017,1,240,60,60,5,0,0,TAB,...,217.86,108.93,120.0,0.0,4,16,22,60,3,1
4,07-08-2017,01-09-2017,1,90,30,90,3,0,0,TAB,...,0.00,0.00,30.0,0.0,8,32,6,90,3,0


In [9]:
test_data['orderdate'] =  pd.to_datetime(test_data['orderdate'], format='%d-%m-%Y')
test_data['filldate'] =  pd.to_datetime(test_data['filldate'], format='%d-%m-%Y')
test_data['MedArrDur']= (test_data.filldate - test_data.orderdate).astype('timedelta64[D]').astype('int')

C:\Users\dell\AppData\Local\Temp/ipykernel_12460/251507664.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['orderdate'] =  pd.to_datetime(test_data['orderdate'], format='%d-%m-%Y')
C:\Users\dell\AppData\Local\Temp/ipykernel_12460/251507664.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['filldate'] =  pd.to_datetime(test_data['filldate'], format='%d-%m-%Y')
C:\Users\dell\AppData\Local\Temp/ipykernel_12460/251507664.py:3: SettingWithCopyWarning: 
A value is trying to be set on a c

In [10]:
import numpy as np
object_col=test_data.select_dtypes(include=['object']).columns
for i in object_col:
  test_data[i].replace(np.nan,'NA',inplace=True)

c:\Users\dell\anaconda3\lib\site-packages\pandas\core\generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [11]:
drop_col = ['orderdate','filldate']
test_data.drop(drop_col,axis=1,inplace=True)

c:\Users\dell\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [12]:
columns = ['HouseholdIncome']
test_data[columns] = test_data[columns].replace(' N ',0)
test_data[columns] = test_data[columns].replace('-',0)
test_data[columns] = test_data[columns].replace(' - ',0)
test_data[columns] = test_data[columns].replace('NA',0)
test_data[columns] = test_data[columns].astype(float)

c:\Users\dell\anaconda3\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [13]:
X = test_data.drop("miss_flag", axis=1)
y_orig=test_data['miss_flag']

In [14]:
X.head()

,dispensedqty1,rxqty,daysupp,refills,rxstatus,deliverystatus,drug_form,routeofadmin,sex,age,...,monthlyCoPay,UnitsPerMOnth,AffordabilityIndex,OrderMonth,orderweek,DaysWithNoMedicine,totalSupply,totalRefills,abandonDays,MedArrDur
0,120,30,40,5,0,0,TAB,Intravenous,M,30.750000,...,62.64,90.0,0.0,4,16,32,40,3,1,0
1,300,60,150,5,0,0,CAP,NOT SPECIFIED,M,30.750000,...,225.00,60.0,0.0,4,16,0,150,3,0,0
2,4,1,4,3,0,0,SOSY,Subcutaneous,F,37.666666,...,198037.50,30.0,0.0,3,9,87,4,4,0,3
3,240,60,60,5,0,0,TAB,NOT SPECIFIED,M,30.750000,...,108.93,120.0,0.0,4,16,22,60,3,1,0
4,90,30,90,3,0,0,TAB,NOT SPECIFIED,F,57.333333,...,0.00,30.0,0.0,8,32,6,90,3,0,25


In [15]:
from shapash import SmartExplainer

In [16]:
xpl = SmartExplainer(
    model=pred_model,

)

In [17]:
xpl.compile(x=X,
             y_target=y_orig # Optional: allows to display True Values vs Predicted Values
           )

In [18]:
app = xpl.run_app(title_story='Dashboard', port=8020)

c:\Users\dell\anaconda3\lib\site-packages\shapash\webapp\smart_app.py:354: FutureWarning:

Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.



Dash is running on http://0.0.0.0:8020/



INFO:root:Your Shapash application run on http://DESKTOP-FLNUFPO:8020/
INFO:root:Use the method .kill() to down your app.
INFO:dash.dash:Dash is running on http://0.0.0.0:8020/



 * Serving Flask app "shapash.webapp.smart_app" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://192.168.0.208:8020/ (Press CTRL+C to quit)
